In [110]:
import pandas as pd
import random
import hashlib
import socket
import threading
import pickle

In [111]:
df = pd.read_csv('unique_ids.csv')
national_ids = df['id'].tolist()


valid_ids = random.sample(national_ids, 1000)

In [112]:
valid_ids

[539431,
 712008,
 476475,
 500606,
 215353,
 740443,
 685958,
 128827,
 925001,
 568460,
 279651,
 825013,
 748046,
 754036,
 522141,
 721770,
 305695,
 148748,
 340729,
 852810,
 822852,
 135483,
 340601,
 733551,
 779864,
 607102,
 980925,
 865378,
 753781,
 167257,
 468307,
 291092,
 680261,
 373132,
 970384,
 544389,
 816124,
 977794,
 639685,
 982542,
 374228,
 314735,
 361187,
 828473,
 872496,
 930212,
 561588,
 773785,
 273572,
 476333,
 256561,
 812985,
 511088,
 825378,
 776244,
 499965,
 479561,
 242190,
 650142,
 898133,
 971825,
 634186,
 979385,
 636283,
 589873,
 136564,
 460881,
 913473,
 966112,
 575787,
 589476,
 347201,
 716836,
 259885,
 976508,
 471588,
 387187,
 682204,
 421099,
 342467,
 792398,
 949787,
 182803,
 670591,
 464544,
 465156,
 849791,
 820673,
 993016,
 394627,
 953437,
 844361,
 216555,
 287115,
 288025,
 985387,
 474119,
 706139,
 159686,
 331043,
 299893,
 249655,
 620670,
 285934,
 989336,
 136625,
 774814,
 580050,
 900640,
 571912,
 788212,
 

In [113]:
invalid_ids = []
while len(invalid_ids) < 1000:
    fake_id = str(random.randint(1000000000, 9999999999))
    if fake_id not in national_ids:
        invalid_ids.append(fake_id)

In [114]:
valid_ids

[539431,
 712008,
 476475,
 500606,
 215353,
 740443,
 685958,
 128827,
 925001,
 568460,
 279651,
 825013,
 748046,
 754036,
 522141,
 721770,
 305695,
 148748,
 340729,
 852810,
 822852,
 135483,
 340601,
 733551,
 779864,
 607102,
 980925,
 865378,
 753781,
 167257,
 468307,
 291092,
 680261,
 373132,
 970384,
 544389,
 816124,
 977794,
 639685,
 982542,
 374228,
 314735,
 361187,
 828473,
 872496,
 930212,
 561588,
 773785,
 273572,
 476333,
 256561,
 812985,
 511088,
 825378,
 776244,
 499965,
 479561,
 242190,
 650142,
 898133,
 971825,
 634186,
 979385,
 636283,
 589873,
 136564,
 460881,
 913473,
 966112,
 575787,
 589476,
 347201,
 716836,
 259885,
 976508,
 471588,
 387187,
 682204,
 421099,
 342467,
 792398,
 949787,
 182803,
 670591,
 464544,
 465156,
 849791,
 820673,
 993016,
 394627,
 953437,
 844361,
 216555,
 287115,
 288025,
 985387,
 474119,
 706139,
 159686,
 331043,
 299893,
 249655,
 620670,
 285934,
 989336,
 136625,
 774814,
 580050,
 900640,
 571912,
 788212,
 

In [115]:
class BloomFilter:
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = [0] * size

    def _hashes(self, item):
        result = []
        for i in range(self.hash_count):
        
            hash_result = int(hashlib.md5((str(item) + str(i)).encode('utf-8')).hexdigest(), 16) % self.size
            result.append(hash_result)
        return result

    def add(self, item):
        for hash_value in self._hashes(item):
            self.bit_array[hash_value] = 1

    def check(self, item):
        for hash_value in self._hashes(item):
            if self.bit_array[hash_value] == 0:
                return False
        return True


# with 1 hash function and size of 20000

In [116]:
bloom_filter = BloomFilter(size=20000, hash_count=1)


for id in valid_ids:
    bloom_filter.add(id)

In [117]:
len(bloom_filter.bit_array)

20000

In [118]:
def handle_client(client_socket):
    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                break
            national_id = pickle.loads(data)
            if bloom_filter.check(national_id):
                response = "valid"
                
            else:
                response = "invalid"
            client_socket.send(response.encode('utf-8'))
            # print('send',response.encode('utf-8') , "from server")
        except Exception as e:
            print(f"Error: {e}")
            client_socket.close()
            break

In [119]:
def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 1010))
    server_socket.listen(5)
    print("Running server on port 1010")

    while True:
        client_socket, addr = server_socket.accept()
        print(f"Connected from {addr}")
        client_handler = threading.Thread(target=handle_client, args=(client_socket,))
        client_handler.start()

In [120]:
server_thread = threading.Thread(target=server)
server_thread.start()

Running server on port 1010


In [121]:
def client(national_ids):
    valid_count = 0
    invalid_count = 0
    false_positive_count=0
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 1010))

    for national_id in national_ids:
        
        client_socket.send(pickle.dumps(national_id))
        
        response = client_socket.recv(1024).decode('utf-8')

        if(response =='valid'):
           valid_count+=1
           if national_id in invalid_ids:
                false_positive_count += 1
        if(response =='invalid'):
           invalid_count+=1


        print(f"natinal code: {national_id}, is: {response}")
        
    print(f"Number of valid inputs: {valid_count}")
    print(f"Number of invalid inputs: {invalid_count}")
    print(f"Number of false positives: {false_positive_count}")
    client_socket.close()





In [122]:
all_ids = valid_ids + invalid_ids
random.shuffle(all_ids)
client(all_ids)

Connected from ('127.0.0.1', 62012)
natinal code: 173920, is: valid
natinal code: 739539, is: valid
natinal code: 575315, is: valid
natinal code: 675344, is: valid
natinal code: 4642103074, is: invalid
natinal code: 6618525012, is: invalid
natinal code: 4624114369, is: invalid
natinal code: 461589, is: valid
natinal code: 1904081763, is: invalid
natinal code: 9941302983, is: invalid
natinal code: 786010, is: valid
natinal code: 1706546114, is: invalid
natinal code: 5015224770, is: invalid
natinal code: 1363655284, is: valid
natinal code: 970384, is: valid
natinal code: 3798775362, is: invalid
natinal code: 801347, is: valid
natinal code: 3983725361, is: invalid
natinal code: 991571, is: valid
natinal code: 470902, is: valid
natinal code: 182803, is: valid
natinal code: 2289721813, is: valid
natinal code: 204402, is: valid
natinal code: 5154919011, is: invalid
natinal code: 8195196497, is: invalid
natinal code: 1472822796, is: invalid
natinal code: 3336840075, is: invalid
natinal code: 

natinal code: 9455378385, is: invalid
natinal code: 256561, is: valid
natinal code: 9624227258, is: invalid
natinal code: 1651207475, is: invalid
natinal code: 518133, is: valid
natinal code: 671732, is: valid
natinal code: 779431, is: valid
natinal code: 9804871617, is: invalid
natinal code: 9180673990, is: invalid
natinal code: 151546, is: valid
natinal code: 409908, is: valid
natinal code: 384671, is: valid
natinal code: 3636411250, is: invalid
natinal code: 8742551071, is: invalid
natinal code: 759362, is: valid
natinal code: 167652, is: valid
natinal code: 3237628557, is: invalid
natinal code: 1415988831, is: invalid
natinal code: 2490835494, is: invalid
natinal code: 413166, is: valid
natinal code: 671690, is: valid
natinal code: 1472680827, is: invalid
natinal code: 9005109127, is: invalid
natinal code: 3614825693, is: invalid
natinal code: 841174, is: valid
natinal code: 9902968485, is: invalid
natinal code: 4554417327, is: valid
natinal code: 1992973848, is: invalid
natinal co

# increase  number of  hash function 

In [123]:
bloom_filter = BloomFilter(size=20000, hash_count=3)


for id in valid_ids:
    bloom_filter.add(id)

In [124]:
def handle_client(client_socket):
    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                break
            national_id = pickle.loads(data)
            if bloom_filter.check(national_id):
                response = "valid"
                
            else:
                response = "invalid"
            client_socket.send(response.encode('utf-8'))
            # print('send',response.encode('utf-8') , "from server")
        except Exception as e:
            print(f"Error: {e}")
            client_socket.close()
            break

In [125]:
def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 1010))
    server_socket.listen(5)
    print("Running server on port 1010")

    while True:
        client_socket, addr = server_socket.accept()
        print(f"Connected from {addr}")
        client_handler = threading.Thread(target=handle_client, args=(client_socket,))
        client_handler.start()

In [126]:
server_thread = threading.Thread(target=server)
server_thread.start()

Running server on port 1010


In [127]:
def client(national_ids):
    valid_count = 0
    invalid_count = 0
    false_positive_count=0
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 1010))

    for national_id in national_ids:
        
        client_socket.send(pickle.dumps(national_id))
        
        response = client_socket.recv(1024).decode('utf-8')

        if(response =='valid'):
           valid_count+=1
           if national_id in invalid_ids:
                false_positive_count += 1
        if(response =='invalid'):
           invalid_count+=1


        print(f"natinal code: {national_id}, is: {response}")
        
    print(f"Number of valid inputs: {valid_count}")
    print(f"Number of invalid inputs: {invalid_count}")
    print(f"Number of false positives: {false_positive_count}")
    client_socket.close()





In [128]:
all_ids = valid_ids + invalid_ids
random.shuffle(all_ids)
client(all_ids)

Connected from ('127.0.0.1', 62013)
natinal code: 319447, is: valid
natinal code: 9709975148, is: invalid
natinal code: 2169686712, is: invalid
natinal code: 701666, is: valid
natinal code: 376974, is: valid
natinal code: 951966, is: valid
natinal code: 286221, is: valid
natinal code: 307629, is: valid
natinal code: 321994, is: valid
natinal code: 467639, is: valid
natinal code: 672091, is: valid
natinal code: 499965, is: valid
natinal code: 5111192718, is: invalid
natinal code: 809214, is: valid
natinal code: 2417632860, is: invalid
natinal code: 8207926976, is: invalid
natinal code: 4821510309, is: invalid
natinal code: 6721105013, is: invalid
natinal code: 404360, is: valid
natinal code: 7282895059, is: invalid
natinal code: 315470, is: valid
natinal code: 3682212293, is: invalid
natinal code: 9515974032, is: invalid
natinal code: 434282, is: valid
natinal code: 9838398112, is: invalid
natinal code: 571912, is: valid
natinal code: 788924, is: valid
natinal code: 3957840315, is: inva

natinal code: 5261522885, is: invalid
natinal code: 1081094131, is: invalid
natinal code: 3137842835, is: invalid
natinal code: 7235210672, is: invalid
natinal code: 908169, is: valid
natinal code: 852323, is: valid
natinal code: 8720799210, is: invalid
natinal code: 473089, is: valid
natinal code: 7295523047, is: invalid
natinal code: 7602795463, is: invalid
natinal code: 712651, is: valid
natinal code: 639090, is: valid
natinal code: 3521034063, is: invalid
natinal code: 1507762225, is: invalid
natinal code: 463636, is: valid
natinal code: 4607549705, is: invalid
natinal code: 309899, is: valid
natinal code: 3718567006, is: invalid
natinal code: 417006, is: valid
natinal code: 4895531783, is: invalid
natinal code: 7435852000, is: invalid
natinal code: 2821264765, is: invalid
natinal code: 891503, is: valid
natinal code: 5679729783, is: invalid
natinal code: 963621, is: valid
natinal code: 832788, is: valid
natinal code: 196166, is: valid
natinal code: 779431, is: valid
natinal code: 

# increase size of bitmap

In [129]:
bloom_filter = BloomFilter(size=80000, hash_count=1)


for id in valid_ids:
    bloom_filter.add(id)

In [130]:
def handle_client(client_socket):
    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                break
            national_id = pickle.loads(data)
            if bloom_filter.check(national_id):
                response = "valid"
                
            else:
                response = "invalid"
            client_socket.send(response.encode('utf-8'))
            # print('send',response.encode('utf-8') , "from server")
        except Exception as e:
            print(f"Error: {e}")
            client_socket.close()
            break

In [131]:
def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 1010))
    server_socket.listen(5)
    print("Running server on port 1010")

    while True:
        client_socket, addr = server_socket.accept()
        print(f"Connected from {addr}")
        client_handler = threading.Thread(target=handle_client, args=(client_socket,))
        client_handler.start()

In [132]:
server_thread = threading.Thread(target=server)
server_thread.start()

Running server on port 1010


In [133]:
def client(national_ids):
    valid_count = 0
    invalid_count = 0
    false_positive_count=0
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 1010))

    for national_id in national_ids:
        
        client_socket.send(pickle.dumps(national_id))
        
        response = client_socket.recv(1024).decode('utf-8')

        if(response =='valid'):
           valid_count+=1
           if national_id in invalid_ids:
                false_positive_count += 1
        if(response =='invalid'):
           invalid_count+=1


        print(f"natinal code: {national_id}, is: {response}")
        
    print(f"Number of valid inputs: {valid_count}")
    print(f"Number of invalid inputs: {invalid_count}")
    print(f"Number of false positives: {false_positive_count}")
    client_socket.close()





In [134]:
all_ids = valid_ids + invalid_ids
random.shuffle(all_ids)
client(all_ids)

Connected from ('127.0.0.1', 62014)
natinal code: 539431, is: valid
natinal code: 2238748824, is: invalid
natinal code: 436018, is: valid
natinal code: 464208, is: valid
natinal code: 9076541775, is: invalid
natinal code: 644654, is: valid
natinal code: 6451868729, is: invalid
natinal code: 744720, is: valid
natinal code: 196166, is: valid
natinal code: 907615, is: valid
natinal code: 6618525012, is: invalid
natinal code: 889202, is: valid
natinal code: 457512, is: valid
natinal code: 5586313215, is: invalid
natinal code: 8179920241, is: invalid
natinal code: 5013455660, is: invalid
natinal code: 175588, is: valid
natinal code: 1468766018, is: invalid
natinal code: 6777097286, is: invalid
natinal code: 753148, is: valid
natinal code: 2639131806, is: invalid
natinal code: 474119, is: valid
natinal code: 691295, is: valid
natinal code: 6457054657, is: invalid
natinal code: 9351912807, is: invalid
natinal code: 7187245246, is: invalid
natinal code: 7974317407, is: invalid
natinal code: 88